# Databases

## Introduction

A huge amount of data lives in databases, so it's essential that you know how to access them. Sometimes you can ask someone to download a snapshot into a .csv for you, but this gets painful quickly: every time you need to make a change you'll have to communicate with another human. You want to be able to reach into the database directly to get the data you need, when you need it.

In this chapter, you'll first learn the basics of Python packages that can interact with SQL databases.

## Connecting to Databases

To connect to a database from Python, there are a number of options, but it does depend on what *kind* of database you are connecting to. The steps (eg connect to the database, upload data, run SQL series) are similar though, so we won't lose much by just choosing one example.

SQLite is a small, fast, self-contained, SQL database engine and the most used database engine in the world. A lot of the data on your computer and mobile phone will in fact be in SQLite databases.

We will always need two steps though, regardless of what kind of SQL database we are connecting to:

- You'll always use a database interface that provides a connection to the database, for example Python's built-in **sqlite** package.
- You'll also use a package that pushes and/or pulls data to/from the database, for example **pandas**.


## Working with Databases Directly

Let's connect to a small SQLite database called the [Chinook database](https://github.com/lerocha/chinook-database), which contains information about the artists, songs, and albums from a music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in eleven tables. The figure below shows the data schema:

For starters, let's use Python's built in **sqlite3** engine to connect to the database and execute a very simple SQL query to select the first ten entries from the 'Artists' table:

In [1]:
import sqlite3

import pandas as pd


In [2]:
# We need to firt download the .sqlite database file
!gdown "1Pedy3vxwYWFOJjADGYBRN2IA95glpIfF"

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\afeli\anaconda3\Scripts\gdown.exe\__main__.py", line 7, in <module>
  File "C:\Users\afeli\anaconda3\Lib\site-packages\gdown\cli.py", line 156, in main
    filename = download(
               ^^^^^^^^^
  File "C:\Users\afeli\anaconda3\Lib\site-packages\gdown\download.py", line 259, in download
    filename_from_url = m.groups()[0]
                        ^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'groups'


In [3]:
# We can establish a connection to the database with this command
con = sqlite3.connect("Chinook_Sqlite.sqlite")


In [4]:
# List all the tables
# First we must type the SQL command and execute it with execute()
cursor = con.execute(
  """
  SELECT name
  FROM sqlite_master
  WHERE type='table';
  """
)

# Then we can retreive the results using fetchall()
rows = cursor.fetchall()
rows

[]

In [5]:
# Let's see another example
# Listing only 10 artists
cursor = con.execute(
  """
  SELECT *
  FROM Artist
  LIMIT 10;
  """
)

rows = cursor.fetchall()
rows

OperationalError: no such table: Artist

Note that the output here is in the form a Python object called a tuple. If we wanted to put this into a **pandas** dataframe, we can just pass it straight in:

In [6]:
import pandas as pd

pd.DataFrame(rows)

""


Another useful hint is that if you're not sure what the column names are, you can obtain them from:

In [7]:
[i[0] for i in cursor.description]

['name']

Putting all togheter...

In [8]:
pd.DataFrame(rows, columns=[i[0] for i in cursor.description])

,name


### Creating a Database

Often, you'll want to create a SQL database to later (efficiently) access cuts of data. Let's create a test database directly using the **sqlite** package. This process involves a `CREATE TABLE` statement, then the name of the table followed by the names of the columns and their data types.

In [9]:
create_query = """CREATE TABLE test (country VARCHAR(20), gdp REAL, health INTEGER);"""
con_new = sqlite3.connect("test_database.sqlite")
con_new.execute(create_query)
con_new.commit()

If this runs, then you've created a test database! (You can check the data directory on your own computer to see if it worked, but you'll get an error if the data directory doesn't already exist.)

Let's now fill the database with some values:

In [10]:
test_data = [("US", 1, 3), ("UK", 0.6, 2), ("France", 0.8, 1)]

con_new.executemany("INSERT INTO test VALUES(?, ?, ?)", test_data)
con_new.commit()

Finally, let's check if this has worked:

In [11]:
con_new.execute("SELECT * FROM test").fetchall()

[('US', 1.0, 3), ('UK', 0.6, 2), ('France', 0.8, 1)]

## SQL with **pandas**

**Pandas** is well-equipped for working with SQL. We can simply push the query we just created straight through using its `read_sql` function—but bear in mind we need to pass in the connection we created to the database too:

In [12]:
con = sqlite3.connect("Chinook_Sqlite.sqlite")

In [13]:
sql_top10_artists = """
  SELECT *
  FROM Artist
  LIMIT 10;
"""

df_top10_artists = pd.read_sql(sql_top10_artists, con)
df_top10_artists

DatabaseError: Execution failed on sql '
  SELECT *
  FROM Artist
  LIMIT 10;
': no such table: Artist

One nice feature of this is that the column names in SQL get passed straight to the column names in our dataframe.

### Creating a SQL Table from Pandas Dataframe

Now I will use the table above to show how you can also create your own tables in the SQLite Database using pandas:


In [14]:
df_top10_artists.to_sql('Top10_Artist', con)


NameError: name 'df_top10_artists' is not defined

In [ ]:
# List all the tables, Top10_Artist should be included now
sql_list_tables = """
  SELECT name
  FROM sqlite_master
  WHERE type='table';
  """

pd.read_sql(sql_list_tables, con)


### Examples of SQL queries with Pandas

In [ ]:
sql_query = """
-- SELECT example: Select InvoiceId, BillingAddress and BillingCity columns from Invoice table. (This is a comment and won’t be executed)
SELECT InvoiceId, BillingAddress, BillingCity
FROM Invoice;
"""

pd.read_sql(sql_query, con)

In [ ]:
sql_query = """
-- WHERE example: Select all the invoices billed from Argentina in which the spent value is over $5
SELECT *
FROM Invoice
WHERE Invoice.BillingCountry = 'Argentina' AND Invoice.Total > 5;
"""

pd.read_sql(sql_query, con)

In [ ]:
sql_query = """
-- More complex example with alias
SELECT *
FROM Customer as c, Invoice AS i
WHERE c.FirstName = 'Mark' AND c.CustomerId = i.CustomerId;
"""

pd.read_sql(sql_query, con)

In [ ]:
sql_query = """
-- GROUP BY example: Number of sales and average amount spent by Country during 2010
SELECT COUNT(*) as Sales_by_Country, AVG(i.Total) as Average_spent, i.BillingCountry as Country
FROM Invoice AS i
WHERE i.InvoiceDate BETWEEN '2010-01-01 00:00:00' and '2010-31-12 00:00:00'
GROUP BY i.BillingCountry
ORDER BY Sales_by_Country DESC;
"""

pd.read_sql(sql_query, con)


In [ ]:
sql_query = """
-- GROUP BY example: Number of sales and average amount spent by Country, only for those who had more than 10 sales
SELECT COUNT(*) as Sales_by_Country, AVG(i.Total) as Average_spent, i.BillingCountry as Country
FROM Invoice AS i
GROUP BY i.BillingCountry
HAVING Sales_by_Country > 10
ORDER BY Sales_by_Country DESC;
"""

pd.read_sql(sql_query, con)

In [ ]:
sql_query = """
-- CASE + GRUOP BY example: Count Tracks by Quality
SELECT case_result.AudioQuality, COUNT(*) FROM (
    SELECT *,
    (CASE
    	WHEN Track.Bytes < 2000000 THEN 'Poor-Quality'
        WHEN Track.Bytes >= 2000000 AND Track.Bytes <= 8000000 THEN 'Mid-Quality'
        WHEN Track.Bytes > 8000000 THEN 'High-Quality'
    END) AS AudioQuality
    FROM Track
    ORDER BY Track.Bytes ASC
) AS case_result
GROUP BY case_result.AudioQuality;
"""

pd.read_sql(sql_query, con)

In [ ]:
sql_query = """
-- INNER JOIN: Employee asigned to each Customer
SELECT c.CustomerId, e.EmployeeId, e.FirstName, e.LastName, e.Title
FROM Customer AS c
INNER JOIN Employee as e
ON c.SupportRepId = e.EmployeeId;
"""

pd.read_sql(sql_query, con)